<a href="https://colab.research.google.com/github/PuchToTalk/ConsumerEdgeHackaton/blob/data/Data_extraction_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install wikipedia-api


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=6fa1c97acafe36e99cff1d0d548ea5ba4581407769e9b52d58e75c9157ab88fe
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


## Simple Extraction from Wikipedia


In [10]:
import requests
import csv
from bs4 import BeautifulSoup

def get_wikipedia_data_csv(title, language="en"):
    # URL de l'API Wikipedia
    url = f"https://{language}.wikipedia.org/w/api.php"

    # Paramètres pour extraire les différentes informations de la page
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts|links|categories|images|info",
        "exintro": True,
        "titles": title,
        "pllimit": "max",
        "cllimit": "max",
        "inprop": "url"
    }

    # Requête à l'API Wikipedia
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        page_id = next(iter(data['query']['pages']))
        if page_id == "-1":
            print(f"La page '{title}' n'existe pas.")
        else:
            page = data['query']['pages'][page_id]

            # Extraction des données de l'article
            page_title = page.get('title', 'N/A')
            page_url = page.get('fullurl', 'N/A')

            # Extraire et nettoyer le texte de l'article
            raw_extract = page.get('extract', 'N/A')
            if raw_extract != 'N/A':
                soup = BeautifulSoup(raw_extract, "html.parser")
                article_extract = soup.get_text(strip=True)  # Supprime toutes les balises HTML
            else:
                article_extract = 'N/A'

            # Calcul du nombre de mots dans l'extrait nettoyé
            word_count = len(article_extract.split()) if article_extract != 'N/A' else 0

            # Extraction des liens internes
            internal_links = "; ".join([link['title'] for link in page.get('links', []) if 'title' in link])

            # Extraction des catégories
            categories = "; ".join([category['title'] for category in page.get('categories', []) if 'title' in category])

            # Extraction des images (nom de fichier seulement)
            images = "; ".join([image['title'] for image in page.get('images', []) if 'title' in image])

            # Écriture dans le fichier CSV
            with open('wikipedia_data.csv', mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([page_title, page_url, article_extract, word_count, internal_links, categories, images])
    else:
        print("Erreur lors de la récupération des données.")

# Initialiser le fichier CSV avec les en-têtes
with open('wikipedia_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"])

# Appeler la fonction pour obtenir les détails de la page "Christmas" et les enregistrer dans un fichier CSV
get_wikipedia_data_csv("Christmas")
get_wikipedia_data_csv("Artificial intelligence")


# Initial Dataset from list of children's stories

In [11]:
import requests
import csv

def verify_wikipedia_pages(titles, language="fr"):
    # URL de base de l'API Wikipedia
    base_url = f"https://{language}.wikipedia.org/w/api.php"

    # Initialiser le fichier CSV avec les résultats
    with open('wikipedia_pages_verification.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["title", "exists", "wikipedia_url"])

        # Vérifier chaque titre
        for title in titles:
            params = {
                "action": "query",
                "format": "json",
                "titles": title
            }

            # Requête à l'API Wikipedia
            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                data = response.json()
                page_id = next(iter(data['query']['pages']))

                if page_id == "-1":
                    # La page n'existe pas
                    writer.writerow([title, False, "N/A"])
                else:
                    # La page existe
                    page_url = f"https://{language}.wikipedia.org/wiki/{title.replace(' ', '_')}"
                    writer.writerow([title, True, page_url])
            else:
                print(f"Erreur lors de la vérification du titre '{title}'.")

# Liste des titres à vérifier
titles = [
    "Alice au Pays des Merveilles", "Peter Pan", "Robin des Bois", "Mulan", "Pocahontas",
    "Le Livre de la Jungle", "Bambi", "Le Roi Lion", "Les Aventures de Winnie l’Ourson",
    "La Princesse et la Grenouille", "La Reine des Neiges", "Moana (Vaiana)", "Blanche-Neige et Rose-Rouge",
    "Ali Baba et les Quarante Voleurs", "Sinbad le Marin", "Le Petit Poucet", "Peau d’Âne",
    "Les Aventures de Tintin", "Astérix et Obélix", "Le Magicien d’Oz", "Oliver Twist",
    "Les Douze Princesses Dansantes", "Le Joueur de Flûte de Hamelin", "La Princesse au Petit Pois",
    "Les Musiciens de Brême", "Ali et le Génie", "Anastasia", "La Petite Fille aux Allumettes",
    "Les Cygnes Sauvages", "Madame Bovary (version simplifiée)", "Heidi", "L’Étrange Noël de Monsieur Jack",
    "Wallace et Gromit", "Chicken Run", "Madagascar", "Kung Fu Panda", "Toy Story",
    "Le Monde de Nemo", "Les Indestructibles", "Shrek", "Cars", "Ratatouille", "Monstres et Cie",
    "Coco", "Les Aventures de Tom Sawyer", "Les Quatre Filles du Docteur March", "Mary Poppins",
    "La Mélodie du Bonheur", "Pippi Longstocking (Fifi Brindacier)", "Peter et Elliott le Dragon",
    "Ferdinand", "Le Petit Prince", "Taram et le Chaudron Magique", "Balto", "Spirit, l’Étalon des Plaines",
    "Les Aristochats", "Le Renard et l’Enfant", "Les Animaux du Bois de Quat’sous", "Bernard et Bianca",
    "Tinker Bell", "Frère des Ours", "Lilo et Stitch", "Rio", "Brave (Rebelle)", "Horton Hears a Who!",
    "Les Schtroumpfs", "Zootopia", "Les Trolls", "La Garde du Roi Lion", "La Légende de Korra",
    "Le Petit Nicolas", "Alice de l’Autre Côté du Miroir", "Charlie et la Chocolaterie", "Le Géant de Fer",
    "Mia et le Migou", "Les Enfants Loups, Ame et Yuki", "Ponyo sur la falaise", "Mon voisin Totoro",
    "Kiki la petite sorcière", "Le Voyage de Chihiro", "Princesse Mononoké", "Le Château ambulant",
    "Nausicaä de la vallée du vent", "Les Enfants de la mer", "Le Château dans le ciel", "Coraline",
    "Anastasia", "Ernest et Célestine", "Paddington", "Le Gruffalo", "Max et les Maximonstres",
    "Le Roi et l’Oiseau", "Kirikou et la Sorcière", "Azur et Asmar", "L’Histoire sans Fin",
    "La Fabuleuse Gilly Hopkins", "Un Chant de Noël", "Le Secret de la Petite Sirène", "Les Trois Brigands",
    "Le Renard et l’Érable"
]

# Vérifier chaque page Wikipedia
verify_wikipedia_pages(titles)


In [13]:
import os

# Get the current working directory
current_path = os.getcwd()
print(current_path)


/content


## Fixing missing values

In [18]:
import csv

# URLs fournis à ajouter
additional_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Winnie_l%27ourson",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/La_Reine_des_neiges",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Le_Bal_des_douze_princesses",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Le_Joueur_de_fl%C3%BBte_de_Hamelin",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/La_Princesse_au_petit_pois",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Aladdin_(Disney)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/La_Petite_Fille_aux_allumettes",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Cygnes_sauvages_(Andersen)",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Madame_Bovary",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/L%27%C3%89trange_No%C3%ABl_de_monsieur_Jack",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/La_M%C3%A9lodie_du_bonheur_(film,_1965)",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fifi_Brindacier",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Taram_et_le_Chaudron_magique",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Spirit,_l%27%C3%A9talon_des_plaines",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Le_Renard_et_l%27Enfant",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/Rebelle_(film_d%27animation,_2012)",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Alice_de_l%27autre_c%C3%B4t%C3%A9_du_miroir",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Le_G%C3%A9ant_de_fer",
    "Les Enfants Loups, Ame et Yuki": "https://fr.wikipedia.org/wiki/Les_Enfants_loups,_Ame_et_Yuki",
    "Le Roi et l’Oiseau": "https://fr.wikipedia.org/wiki/Le_Roi_et_l%27Oiseau",
    "L’Histoire sans Fin": "https://fr.wikipedia.org/wiki/L%27Histoire_sans_fin_(film)",
    "Le Renard et l’Érable": "https://fr.wikipedia.org/wiki/Le_Renard_et_les_Raisins",
    "Moana (Vaiana)" : "https://fr.wikipedia.org/wiki/Vaiana_:_La_L%C3%A9gende_du_bout_du_monde"
    "Coco" : "https://fr.wikipedia.org/wiki/Coco_(film,_2017)"
}

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/wikipedia_pages_verification.csv'
output_csv_path = '/content/wikipedia_pages_verification_updated_final.csv'

# Mise à jour du fichier CSV avec les URLs supplémentaires
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les URLs supplémentaires si nécessaires
    for row in reader:
        title = row['title']
        if title in additional_urls and (row['wikipedia_url'] == "N/A" or not row['wikipedia_url']):
            row['exists'] = True
            row['wikipedia_url'] = additional_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path


'/content/wikipedia_pages_verification_updated_final.csv'

## Generate the corpus & Merge results

In [21]:
import requests
import csv
from bs4 import BeautifulSoup

# Fonction pour obtenir les détails de Wikipédia
def get_wikipedia_data_csv(title, language="fr"):
    # URL de l'API Wikipedia
    url = f"https://{language}.wikipedia.org/w/api.php"

    # Paramètres pour extraire les différentes informations de la page
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts|links|categories|images|info",
        "exintro": True,
        "titles": title,
        "pllimit": "max",
        "cllimit": "max",
        "inprop": "url"
    }

    # Requête à l'API Wikipedia
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        page_id = next(iter(data['query']['pages']))
        if page_id == "-1":
            print(f"La page '{title}' n'existe pas.")
        else:
            page = data['query']['pages'][page_id]

            # Extraction des données de l'article
            page_title = page.get('title', 'N/A')
            page_url = page.get('fullurl', 'N/A')

            # Extraire et nettoyer le texte de l'article
            raw_extract = page.get('extract', 'N/A')
            if raw_extract != 'N/A':
                soup = BeautifulSoup(raw_extract, "html.parser")
                article_extract = soup.get_text(strip=True)  # Supprime toutes les balises HTML
            else:
                article_extract = 'N/A'

            # Calcul du nombre de mots dans l'extrait nettoyé
            word_count = len(article_extract.split()) if article_extract != 'N/A' else 0

            # Extraction des liens internes
            internal_links = "; ".join([link['title'] for link in page.get('links', []) if 'title' in link])

            # Extraction des catégories
            categories = "; ".join([category['title'] for category in page.get('categories', []) if 'title' in category])

            # Extraction des images (nom de fichier seulement)
            images = "; ".join([image['title'] for image in page.get('images', []) if 'title' in image])

            # Retourner les données sous forme de dictionnaire
            return {
                "page_title": page_title,
                "page_url": page_url,
                "article_extract": article_extract,
                "word_count": word_count,
                "internal_links": internal_links,
                "categories": categories,
                "images": images
            }
    else:
        print("Erreur lors de la récupération des données.")
        return None

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/wikipedia_pages_verification_updated_final.csv'
output_csv_path = '/content/wikipedia_data_enriched.csv'

# Lire le fichier CSV et appliquer la fonction à chaque titre
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Appliquer la fonction à chaque ligne
    for row in reader:
        title = row['title']
        if row['exists'].lower() == 'true':
            wikipedia_data = get_wikipedia_data_csv(title)
            if wikipedia_data:
                writer.writerow(wikipedia_data)

# Chemin du fichier CSV enrichi
output_csv_path


La page 'Les Aventures de Winnie l’Ourson' n'existe pas.
La page 'La Reine des Neiges' n'existe pas.
La page 'Moana (Vaiana)' n'existe pas.
La page 'Les Douze Princesses Dansantes' n'existe pas.
La page 'Le Joueur de Flûte de Hamelin' n'existe pas.
La page 'La Princesse au Petit Pois' n'existe pas.
La page 'Ali et le Génie' n'existe pas.
La page 'La Petite Fille aux Allumettes' n'existe pas.
La page 'Les Cygnes Sauvages' n'existe pas.
La page 'Madame Bovary (version simplifiée)' n'existe pas.
La page 'L’Étrange Noël de Monsieur Jack' n'existe pas.
La page 'La Mélodie du Bonheur' n'existe pas.
La page 'Pippi Longstocking (Fifi Brindacier)' n'existe pas.
La page 'Taram et le Chaudron Magique' n'existe pas.
La page 'Spirit, l’Étalon des Plaines' n'existe pas.
La page 'Le Renard et l’Enfant' n'existe pas.
La page 'Brave (Rebelle)' n'existe pas.
La page 'Alice de l’Autre Côté du Miroir' n'existe pas.
La page 'Le Géant de Fer' n'existe pas.
La page 'Les Enfants Loups, Ame et Yuki' n'existe p

'/content/wikipedia_data_enriched.csv'

In [31]:
import csv

# URLs fournis à ajouter
replacement_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Les_Aventures_d%27Alice_au_pays_des_merveilles",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/Le_Livre_de_la_jungle",
    "Moana (Vaiana)": "https://fr.wikipedia.org/wiki/Le_Roi_lion_(film,_1994)",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Peau_d%27Âne",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Astérix",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/Le_Magicien_d%27Oz",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Shrek_(film)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Quatre_Filles_du_docteur_March",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Peter_et_Elliott_le_dragon",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/Les_Animaux_du_Bois_de_Quat%27sous",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/Les_Aventures_de_Bernard_et_Bianca",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fée_Clochette",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Frère_des_ours",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Horton_(film)",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Zootopie",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/La_Garde_du_Roi_lion",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Un_chant_de_Noël",
    "Les Trolls": "https://fr.wikipedia.org/wiki/Les_Trolls_(film)",
    "Les Enfants de la mer": "https://fr.wikipedia.org/wiki/Les_Enfants_de_la_mer_(film)",
    "Nausicaä de la Vallée du Vent": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Ratatouille":"https://fr.wikipedia.org/wiki/Ratatouille_(film)",
    "Cars": "https://fr.wikipedia.org/wiki/Cars_(film)",
    "Shrek": "https://fr.wikipedia.org/wiki/Shrek_(film)",
    "Coco": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)"
}

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/final-wiki.csv'
output_csv_path = '/content/wiki1.csv'

# Mise à jour du fichier CSV avec les URLs de remplacement
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les URLs de remplacement si nécessaire
    for row in reader:
        title = row['title']
        if title in replacement_urls:
            row['exists'] = True
            row['wikipedia_url'] = replacement_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path

'/content/wiki1.csv'

In [ ]:
import csv
import sys

# Augmenter la limite maximale de taille de champ
csv.field_size_limit(sys.maxsize)

# URLs fournies à ajouter ou mettre à jour
new_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Winnie_l%27ourson",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/La_Reine_des_neiges",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Le_Bal_des_douze_princesses",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Le_Joueur_de_fl%C3%BBte_de_Hamelin",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/La_Princesse_au_petit_pois",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Aladdin_(Disney)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/La_Petite_Fille_aux_allumettes",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Cygnes_sauvages_(Andersen)",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Madame_Bovary",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/L%27%C3%89trange_No%C3%ABl_de_monsieur_Jack",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/La_M%C3%A9lodie_du_bonheur_(film,_1965)",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fifi_Brindacier",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Taram_et_le_Chaudron_magique",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Spirit,_l%27%C3%A9talon_des_plaines",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Le_Renard_et_l%27Enfant",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/Rebelle_(film_d%27animation,_2012)",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Alice_de_l%27autre_c%C3%B4t%C3%A9_du_miroir",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Le_G%C3%A9ant_de_fer",
    "Les Enfants Loups, Ame et Yuki": "https://fr.wikipedia.org/wiki/Les_Enfants_loups,_Ame_et_Yuki",
    "Le Roi et l’Oiseau": "https://fr.wikipedia.org/wiki/Le_Roi_et_l%27Oiseau",
    "L’Histoire sans Fin": "https://fr.wikipedia.org/wiki/L%27Histoire_sans_fin_(film)",
    "Le Renard et l’Érable": "https://fr.wikipedia.org/wiki/Le_Renard_et_les_Raisins",
    "Moana (Vaiana)": "https://fr.wikipedia.org/wiki/Vaiana_:_La_L%C3%A9gende_du_bout_du_monde",
    "Coco": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)",
    "Les Trolls": "https://fr.wikipedia.org/wiki/Les_Trolls_(film)",
    "Les Enfants de la mer": "https://fr.wikipedia.org/wiki/Les_Enfants_de_la_mer_(film)",
    "Nausicaä de la Vallée du Vent": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Ratatouille": "https://fr.wikipedia.org/wiki/Ratatouille_(film)",
    "Cars": "https://fr.wikipedia.org/wiki/Cars_(film)",
    "Shrek": "https://fr.wikipedia.org/wiki/Shrek_(film)"
}

# URLs supplémentaires à mettre à jour
additional_urls = {
    "Bambi": "https://fr.wikipedia.org/wiki/Bambi_(film,_1942)",
    "Le Livre de la jungle": "https://fr.wikipedia.org/wiki/Le_Livre_de_la_jungle_(film,_1967)",
    "Anastasia": "https://fr.wikipedia.org/wiki/Anastasia_(film,_1997)",
    "Heidi": "https://fr.wikipedia.org/wiki/Heidi_(série_télévisée_d%27animation)",
    "Mary Poppins": "https://fr.wikipedia.org/wiki/Mary_Poppins_(film)",
    "Les Quatre Filles du docteur March (homonymie)": "https://fr.wikipedia.org/wiki/Les_Quatre_Filles_du_docteur_March",
    "Les Trois Brigands": "https://fr.wikipedia.org/wiki/Les_Trois_Brigands_(film)"
}

# Fusionner toutes les nouvelles URLs
all_urls = {**new_urls, **additional_urls}

# Chemin du fichier CSV téléchargé
input_csv_path = '/mnt/data/first-result.csv'
output_csv_path = '/mnt/data/first-result-overwritten-final-complete.csv'

# Mise à jour du fichier CSV avec les nouvelles URLs en utilisant la colonne 'page_title' pour remplacer la colonne 'page_url'
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les nouvelles URLs pour 'page_url'
    for row in reader:
        title = row['page_title']
        if title in all_urls:
            row['page_url'] = all_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path

In [35]:
from google.colab import files
import requests
import csv
import sys
from bs4 import BeautifulSoup

# Étape 1: Charger le fichier CSV
uploaded = files.upload()  # Téléchargez votre fichier CSV ici.

# Étape 2: Mettre à jour le chemin du fichier CSV avec le nom du fichier téléchargé
input_csv_path = '/content/step1-update.csv'  # Remplacez par le nom de votre fichier téléchargé
output_csv_path = '/content/final-step1.csv'

# Augmenter la limite maximale de taille de champ
csv.field_size_limit(sys.maxsize)

# Étape 3: Fonction pour extraire les détails de Wikipédia
def get_wikipedia_data_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        # Utiliser BeautifulSoup pour extraire le contenu de la page Wikipedia
        soup = BeautifulSoup(response.text, "html.parser")

        # Extraire le titre de la page
        page_title = soup.find("h1", {"id": "firstHeading"}).text if soup.find("h1", {"id": "firstHeading"}) else "N/A"

        # Extraire le texte de l'article (principalement à partir de balises <p>)
        paragraphs = soup.find_all("p")
        article_extract = " ".join([p.text for p in paragraphs])
        article_extract = article_extract.strip()

        # Calcul du nombre de mots dans l'extrait
        word_count = len(article_extract.split()) if article_extract else 0

        # Extraire les liens internes (balises <a> dans le corps de l'article)
        internal_links = []
        for link in soup.find_all("a", href=True):
            href = link.get("href")
            if href.startswith("/wiki/") and not href.startswith("/wiki/Special:"):
                internal_links.append(link.text.strip())
        internal_links_str = "; ".join(internal_links)

        # Extraire les catégories (depuis la section des catégories)
        categories = []
        for category in soup.find_all("div", {"id": "mw-normal-catlinks"}):
            for link in category.find_all("a"):
                categories.append(link.text.strip())
        categories_str = "; ".join(categories)

        # Extraire les images
        images = []
        for img in soup.find_all("img"):
            images.append(img.get("src"))
        images_str = "; ".join(images)

        # Retourner les données sous forme de dictionnaire
        return {
            "page_title": page_title,
            "page_url": url,
            "article_extract": article_extract,
            "word_count": word_count,
            "internal_links": internal_links_str,
            "categories": categories_str,
            "images": images_str
        }
    else:
        print(f"Erreur lors de la récupération de la page : {url}")
        return None

# Étape 4: Lire le fichier CSV et appliquer la fonction à chaque URL
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Appliquer la fonction à chaque ligne
    for row in reader:
        url = row['page_url']
        if url and url != "N/A":
            wikipedia_data = get_wikipedia_data_from_url(url)
            if wikipedia_data:
                writer.writerow(wikipedia_data)

# Étape 5: Télécharger le fichier CSV enrichi
files.download(output_csv_path)

Saving step1-update.csv to step1-update (2).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now that we have the Data "final-step1.csv", we use it to generate the concise text in another column using OpenAI API